<a href="https://colab.research.google.com/github/talhaanwarch/EEG_mental_task/blob/master/augmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install mne
!pip install wfdb

     |████████████████████████████████| 6.6MB 4.1MB/s 
     |████████████████████████████████| 102kB 4.3MB/s 
     |████████████████████████████████| 163kB 9.2MB/s 
  Created wheel for wfdb: filename=wfdb-2.2.1-cp36-none-any.whl size=100368 sha256=1f67725fe26421333cfa709d1b50b2626ec25d0847101c7bc998b057951ea6a5
  Stored in directory: /root/.cache/pip/wheels/bb/a9/00/0078d26b0c15b31be0001af8eb659496709c361c69641303f1
Successfully built wfdb


In [2]:
!pip install tsaug

In [3]:
import wfdb
dbs = wfdb.get_dbs()
wfdb.dl_database('eegmat','data')

Created local base download directory: data
Finished downloading files


In [0]:

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import progressbar
import re
import mne
from scipy import stats

In [5]:
import os
import glob
path = r'/content/data/' # use your path
all_files = glob.glob(os.path.join(path, "*.edf")) 

data1=[]
data2=[]
for filename in (all_files):
    if int(re.findall(r'\d+',filename)[1])==1:
        data1.append(mne.io.read_raw_edf(filename,preload=True).get_data()[0:-3,::]);
    else:
        data2.append(mne.io.read_raw_edf(filename,preload=True).get_data()[0:-3,::]);

Extracting EDF parameters from /content/data/Subject31_1.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 39999  =      0.000 ...    79.998 secs...
Extracting EDF parameters from /content/data/Subject01_2.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 30999  =      0.000 ...    61.998 secs...
Extracting EDF parameters from /content/data/Subject00_2.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 30999  =      0.000 ...    61.998 secs...
Extracting EDF parameters from /content/data/Subject16_1.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 90999  =      0.000 ...   181.998 secs...
Extracting EDF parameters from /content/data/Subject16_2.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 30999  =      0.000 ...    

In [6]:
data1[0].shape

(18, 40000)

In [7]:
data1[0][0].shape

(40000,)

In [0]:

from tsaug import *
my_aug = (
    RandomMagnify(max_zoom=4.0, min_zoom=2.0) * 2
    + RandomTimeWarp() * 2
    + RandomJitter(strength=0.1) @ 0.5
    + RandomTrend(min_anchor=-0.5, max_anchor=0.5) @ 0.5
    + RandomAffine() @ 0.5
    + RandomCrop() * 2
    + RandomCrossSum() @ 0.5
    + RandomSidetrack() @ 0.5
)


In [9]:
my_aug.summary()


Augmentor 	 M 	 Prob 	 Output Size 	 Params
RandomMagnify 	 2 	 1.0 	 (2N, n, c) 	 {'max_zoom': 4.0, 'min_zoom': 2.0, 'random_seed': None}
RandomTimeWarp 	 2 	 1.0 	 (4N, n, c) 	 {'n_speed_change': 3, 'random_seed': None}
RandomJitter 	 1 	 0.5 	 (4N, n, c) 	 {'dist': 'normal', 'strength': 0.1, 'random_seed': None}
RandomTrend 	 1 	 0.5 	 (4N, n, c) 	 {'num_anchors': 5, 'min_anchor': -0.5, 'max_anchor': 0.5, 'random_seed': None}
RandomAffine 	 1 	 0.5 	 (4N, n, c) 	 {'max_a': 10.0, 'min_a': -10.0, 'max_b': 100.0, 'min_b': -100.0, 'random_seed': None}
RandomCrop 	 2 	 1.0 	 (8N, n, c) 	 {'crop_size': None, 'crops_per_series': 1, 'random_seed': None}
RandomCrossSum 	 1 	 0.5 	 (8N, n, c) 	 {'max_sum_series': 5, 'random_seed': None}
RandomSidetrack 	 1 	 0.5 	 (8N, n, c) 	 {'mode': 'multiplicative', 'initial_sidetrack': None, 'max_sidetrack': None, 'min_sidetrack': None, 'step_mu': 0, 'step_sigma': 0.1, 'random_seed': None}


In [0]:
dim=500
def reshape(data):   
    shape=data.ravel().shape[0]
    s=shape%(18*500)
    if s!=0:
        d=data.ravel()[s//2:-s//2].reshape(-1,dim,18)
        #return np.swapaxes(d,1,2)   
        return d
    else: 
        d1=data.reshape(-1,dim,18)
        #return np.swapaxes(d1,1,2)   
        return d1

In [0]:
data_1=[]
for i in data1:
    data_1.append(reshape(my_aug.run(i)))
    

    
    
data_2=[]
for i in data2:
    data_2.append(reshape(my_aug.run(i)))

In [12]:
data_1[0].shape

(640, 500, 18)

In [13]:
len(data_1),len(data_2)

(36, 36)

In [14]:
import keras 
from tensorflow.keras.layers import Conv1D,MaxPool1D,AveragePooling1D,GlobalAveragePooling1D,Dropout,Dense,BatchNormalization,Flatten,LSTM,Input,concatenate,GlobalMaxPooling1D,Bidirectional
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.optimizers import Adam,RMSprop,SGD
#from tensorflow.keras.activations import ReLU
from tensorflow.keras.layers import LeakyReLU
from sklearn.model_selection import StratifiedKFold    
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedKFold
from tensorflow.keras.callbacks import ModelCheckpoint,ReduceLROnPlateau
from sklearn.utils import shuffle
import tensorflow

Using TensorFlow backend.


In [0]:
from tensorflow.keras.callbacks import ReduceLROnPlateau,ModelCheckpoint
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.4,
                              patience=3, min_lr=0.000001, verbose=1)

In [0]:

history=[]
test_accuracy=[]
val_accuracy=[]
for i in range(34):
    HC=data_1.copy()
    SZ=data_2.copy()
    #train, test, val data
    Xtest0=np.array(HC.pop(i))
    Xval0=np.array(HC.pop(i+1))
    Xtrain0=np.concatenate(HC)
    #create labels for them
    ytest0=np.zeros(Xtest0.shape[0])
    yval0=np.zeros(Xval0.shape[0])
    ytrain0=np.zeros(Xtrain0.shape[0])
    
    #create rain, test, val data
    Xtest1=np.array(SZ.pop(i))
    Xval1=np.array(SZ.pop(i+1))
    Xtrain1=np.concatenate(SZ)

    ytest1=np.ones(Xtest1.shape[0])
    yval1=np.ones(Xval1.shape[0])
    ytrain1=np.ones(Xtrain1.shape[0])
    #combined data
    X_test=np.concatenate((Xtest0,Xtest1))
    X_val=np.concatenate((Xval0,Xval1))
    X_train=np.concatenate((Xtrain0,Xtrain1))
    #combined labesl
    y_test=np.concatenate((ytest0,ytest1))
    y_val=np.concatenate((yval0,yval1))
    y_train=np.concatenate((ytrain0,ytrain1))
    
    


    # y_val=tensorflow.keras.utils.to_categorical(y_val)
    # y_test=tensorflow.keras.utils.to_categorical(y_test)
    # y_train=tensorflow.keras.utils.to_categorical(y_train)
    
    scaler=StandardScaler()
    scalers = {}
    for i in range(X_train.shape[2]):
        scalers[i] = StandardScaler()
        X_train[:, :, i] = scalers[i].fit_transform(X_train[:, :, i]) 
    
    for i in range(X_test.shape[2]):
        X_test[:, :, i] = scalers[i].transform(X_test[:, :, i])
    
    for i in range(X_test.shape[2]):
        X_val[:, :, i] = scalers[i].transform(X_val[:, :, i])
        
    X_train,y_train=shuffle(X_train,y_train)
    #X_val,y_val=shuffle(X_val,y_val)
    #X_test,y_test=shuffle(X_test,y_test)

    
    inp=Input(shape=(dim,18))
    x=LSTM(16, activation= 'tanh',recurrent_activation ='sigmoid',recurrent_dropout=0,unroll =False,use_bias =True,return_sequences = True)(inp)
    x1=Conv1D(filters=16,kernel_size=3,strides=1,activation='relu')(x)
    x1=GlobalMaxPooling1D()(x1)
    x2=Conv1D(filters=16,kernel_size=5,strides=1,activation='relu')(x)
    x2=GlobalMaxPooling1D()(x2)

    x3=Conv1D(filters=16,kernel_size=7,strides=1,activation='relu')(x)
    x3=GlobalMaxPooling1D()(x3)

    x=concatenate([x1,x2,x3],axis=1)
   
    #x=Flatten()(x)
    x=Dense(16,activation='relu')(x)
    out=Dense(1,activation='sigmoid')(x)

    model=Model(inp,out)
    model.compile(optimizer=Adam(0.0001), loss='binary_crossentropy', metrics=['accuracy'] )
    
 
    history=model.fit(X_train,y_train,validation_data=(X_val,y_val),epochs=30,batch_size=30,verbose=2,callbacks=[reduce_lr])
    val_accuracy.append(model.evaluate(X_val,y_val)[1])
    test_accuracy.append(model.evaluate(X_test,y_test)[1])

    tensorflow.keras.backend.clear_session()



Epoch 1/30
2211/2211 - 49s - loss: 0.5663 - accuracy: 0.7456 - val_loss: 0.5691 - val_accuracy: 0.7459 - lr: 1.0000e-04
Epoch 2/30
2211/2211 - 49s - loss: 0.5528 - accuracy: 0.7457 - val_loss: 0.5668 - val_accuracy: 0.7459 - lr: 1.0000e-04
Epoch 3/30
2211/2211 - 49s - loss: 0.5504 - accuracy: 0.7456 - val_loss: 0.5655 - val_accuracy: 0.7459 - lr: 1.0000e-04
Epoch 4/30
2211/2211 - 49s - loss: 0.5489 - accuracy: 0.7455 - val_loss: 0.5613 - val_accuracy: 0.7418 - lr: 1.0000e-04
Epoch 5/30
2211/2211 - 49s - loss: 0.5479 - accuracy: 0.7451 - val_loss: 0.5601 - val_accuracy: 0.7454 - lr: 1.0000e-04
Epoch 6/30
2211/2211 - 49s - loss: 0.5466 - accuracy: 0.7456 - val_loss: 0.5608 - val_accuracy: 0.7444 - lr: 1.0000e-04
Epoch 7/30
2211/2211 - 49s - loss: 0.5463 - accuracy: 0.7459 - val_loss: 0.5595 - val_accuracy: 0.7433 - lr: 1.0000e-04
Epoch 8/30
2211/2211 - 49s - loss: 0.5457 - accuracy: 0.7458 - val_loss: 0.5578 - val_accuracy: 0.7459 - lr: 1.0000e-04
Epoch 9/30
2211/2211 - 49s - loss: 0.545

In [0]:
# import gc
# del model
# tensorflow.keras.backend.clear_session()
# gc.collect()


In [0]:
np.array(test_accuracy).mean()

In [0]:
np.array(val_accuracy).mean()